## Random Forest with 5-fold CV, 6 repitition on 100k11F dataset
dataset: 11 features, 100k row subsampled dataset, 80:10:10 train:val:test split 

with optuna early stopping

20221015

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import optuna as opt
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import SimpleImputer
from optuna.samplers import TPESampler

In [2]:
#import data
train_df = pd.read_parquet("100K11F_test_main.parquet.snappy")
val_df = pd.read_parquet("100K11F_val_main.parquet.snappy")
test_df = pd.read_parquet("100K11F_test_main.parquet.snappy")
test_df

,r_id,r_stars,r_stars_square,r_length,u_friends_count,u_review_count,u_month_age,b_stars,b_review_count,r_sen,r_sub,r_rea,r_useful
0,1340676,1,1,305,1,1,37.547287,1.5,19,-0.146854,0.439109,77.13,1
1,941404,4,16,20,2,96,40.146040,4.0,120,0.455000,0.452500,83.52,1
2,1312730,1,1,285,333,24,39.063471,4.0,601,-0.096634,0.519085,79.30,3
3,5155149,4,16,139,273,374,7.853903,2.0,17,0.028750,0.298333,82.95,4
4,3706898,4,16,41,1,5,11.606999,2.5,19,0.447917,0.700000,86.71,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4134229,5,25,95,1,12,34.229801,3.5,58,0.145525,0.305864,89.58,1
9996,4059522,3,9,454,149,361,101.389702,4.0,176,0.009922,0.398143,92.02,3
9997,5567999,3,9,347,352,286,92.625418,2.5,20,0.075301,0.390278,70.16,2
9998,2980845,4,16,175,217,219,54.729315,4.0,70,0.161314,0.429562,89.28,2


In [3]:
X_train, y_train = train_df.drop(['r_useful', 'r_id'], axis=1).values, train_df['r_useful'].values
X_val, y_val = val_df.drop(['r_useful', 'r_id'], axis=1).values, val_df['r_useful'].values
X_test, y_test = test_df.drop(['r_useful', 'r_id'], axis=1).values, test_df['r_useful'].values

In [4]:
# Code reference: With many thx to Brian's XGBboost notebook: 100K32F_XGBoost.ipynb

In [5]:
N_OPTUNA_TRIALS = 50
K_FOLDS = 5 
N_REPS = 6
# early stopping threshold
T_ES = 20 
MODEL_PREFIX = "RF"

# Mean imputation and standardisation
model_pipe = Pipeline([
    ("imp", SimpleImputer()),
    ("ss", StandardScaler()),
    (MODEL_PREFIX, RandomForestRegressor())])

In [6]:
def hp_appender(hp_dict):
    '''Return dictionary where every key has the MODEL_PREFIX__ appended.'''
    new_dict = {}
    for key, val in hp_dict.items():
        new_dict[MODEL_PREFIX + "__" + key] = val
    return new_dict

In [7]:
def optuna_objective(trial, model, X, y):
    print(f"{time.strftime('%H:%M:%S', time.localtime())} | Running Optuna Trial: {trial.number}")
    
    
    hyperparams = {
        "n_estimators": trial.suggest_int('n_estimators', 1, 500, 25),
        "max_depth": trial.suggest_int('max_depth', 1, 500, 25),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 20),
        "max_features" : trial.suggest_categorical("max_features", ['sqrt', 'log2']),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 20), 
        "random_state": 760, 
        "n_jobs": -1
    }
    
    print(f"with hyperparameters: {hyperparams}")
    hyperparams = hp_appender(hyperparams)
    model.set_params(**hyperparams)
    
    
    # Inner CV Loop
    avg_score = -cross_val_score(model, X, y,
        scoring="neg_root_mean_squared_error", cv=KFold(K_FOLDS)).mean()
    print(f"complete! average cv RMSE: {avg_score}")
    return avg_score

In [8]:
class early_stopping_check_callback:
    def __init__(self, threshold):
        self.threshold = threshold

    def __call__(self, study, trial):
        # stop study if the number of consecutive trials with no improvement is
        # at least the threshold.
        if trial.number - study.best_trial.number >= self.threshold:
            print("==== EARLY STOPPING ACTIVATED ====")
            study.stop()

In [9]:
def get_best_model(X, y):
    '''Obtains best hyperparameters using TPE sampling and cross-validation.'''
    study = opt.create_study(direction='minimize', sampler=TPESampler(seed=760))
    study.optimize(
        lambda trial: optuna_objective(trial, model_pipe, X, y),
        n_trials=N_OPTUNA_TRIALS,
        callbacks=[early_stopping_check_callback(T_ES)]) # early stopping
    return study.best_params

In [10]:
def fit_and_score(model, hps, X_train, y_train, X_test, y_test):
    model.set_params(**hp_appender(hps))
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)

    # calculate scores
    rmse = mean_squared_error(y_test, y_preds, squared=False)
    mae = mean_absolute_error(y_test, y_preds)
    return rmse, mae

In [11]:
# actual repeated CV loop

repetition_results = {
    "hp": [],
    "rmse": [],
    "mae": []
}

for rep_i in range(N_REPS):
    print(f"====== Running repetition: {rep_i} ======")
    # shuffle training data. Make sure each iteration uses different seed
    cv_train = train_df.sample(frac=1, random_state=760+rep_i)
    # extract X and y arrays
    cv_X_train, cv_y_train = cv_train.drop(['r_useful', 'r_id'], axis=1).values, cv_train['r_useful'].values
    
    # obtain best hyperparameters via cross-validation and TPE sampling.
    best_params = get_best_model(cv_X_train, cv_y_train)
    
    rmse, mae = fit_and_score(
        model_pipe, best_params,
        cv_X_train, cv_y_train,
        X_val, y_val)

    # save results for this iteration
    repetition_results["rmse"].append(rmse)
    repetition_results["mae"].append(mae)
    repetition_results["hp"].append(best_params)

[I 2022-10-15 18:21:26,250] A new study created in memory with name: no-name-ee3661b0-b0df-4129-aa06-961336a9a5d2


====== Running repetition: 0 ======
18:21:26 | Running Optuna Trial: 0
with hyperparameters: {'n_estimators': 51, 'max_depth': 51, 'min_samples_split': 5, 'max_features': 'log2', 'min_samples_leaf': 18, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-15 18:21:28,370] Trial 0 finished with value: 3.3832933490043535 and parameters: {'n_estimators': 51, 'max_depth': 51, 'min_samples_split': 5, 'max_features': 'log2', 'min_samples_leaf': 18}. Best is trial 0 with value: 3.3832933490043535.


complete! average cv RMSE: 3.3832933490043535
18:21:28 | Running Optuna Trial: 1
with hyperparameters: {'n_estimators': 76, 'max_depth': 451, 'min_samples_split': 14, 'max_features': 'sqrt', 'min_samples_leaf': 14, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-15 18:21:28,968] Trial 1 finished with value: 3.384130386431658 and parameters: {'n_estimators': 76, 'max_depth': 451, 'min_samples_split': 14, 'max_features': 'sqrt', 'min_samples_leaf': 14}. Best is trial 0 with value: 3.3832933490043535.


...

complete! average cv RMSE: 3.367984190528939
18:33:14 | Running Optuna Trial: 37
with hyperparameters: {'n_estimators': 151, 'max_depth': 376, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 15, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-15 18:33:16,106] Trial 37 finished with value: 3.370630952924631 and parameters: {'n_estimators': 151, 'max_depth': 376, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 15}. Best is trial 18 with value: 3.364611662488348.


complete! average cv RMSE: 3.370630952924631
18:33:16 | Running Optuna Trial: 38
with hyperparameters: {'n_estimators': 101, 'max_depth': 451, 'min_samples_split': 12, 'max_features': 'sqrt', 'min_samples_leaf': 14, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-15 18:33:17,301] Trial 38 finished with value: 3.3706018239423825 and parameters: {'n_estimators': 101, 'max_depth': 451, 'min_samples_split': 12, 'max_features': 'sqrt', 'min_samples_leaf': 14}. Best is trial 18 with value: 3.364611662488348.


complete! average cv RMSE: 3.3706018239423825
==== EARLY STOPPING ACTIVATED ====


In [12]:
#final model performance on train+val
X_train_val = np.vstack((X_train, X_val))
y_train_val = np.hstack((y_train, y_val))
print(X_train_val.shape)
print(y_train_val.shape)

rmse = repetition_results['rmse']
best_best_i = np.argmin(rmse) # best = minimum RMSE
hp = repetition_results['hp']
best_best_hp = hp[best_best_i]

rmse, mae = fit_and_score(model_pipe, best_best_hp,
    X_train_val, y_train_val, X_test, y_test)

print(f"Best overall HP:{best_best_hp}")
print(f"Best overall RMSE: {rmse:.4}")
print(f"Best overall MAE: {mae:.4}")

(20000, 11)
(20000,)
Best overall HP:{'n_estimators': 426, 'max_depth': 201, 'min_samples_split': 2, 'max_features': 'sqrt', 'min_samples_leaf': 7}
Best overall RMSE: 2.88
Best overall MAE: 1.278


In [13]:
print(repetition_results)

print("\nRMSE")
rmse = repetition_results['rmse']
print(f"mean: {np.mean(rmse):.4}, stdev: {np.std(rmse):.4}")

print("MAE")
mae = repetition_results['mae']
print(f"mean: {np.mean(mae):.4}, stdev: {np.std(mae):.4}")

{'hp': [{'n_estimators': 176, 'max_depth': 176, 'min_samples_split': 2, 'max_features': 'sqrt', 'min_samples_leaf': 16}, {'n_estimators': 451, 'max_depth': 426, 'min_samples_split': 3, 'max_features': 'log2', 'min_samples_leaf': 8}, {'n_estimators': 176, 'max_depth': 151, 'min_samples_split': 13, 'max_features': 'sqrt', 'min_samples_leaf': 8}, {'n_estimators': 226, 'max_depth': 351, 'min_samples_split': 18, 'max_features': 'sqrt', 'min_samples_leaf': 11}, {'n_estimators': 426, 'max_depth': 201, 'min_samples_split': 2, 'max_features': 'sqrt', 'min_samples_leaf': 7}, {'n_estimators': 126, 'max_depth': 401, 'min_samples_split': 2, 'max_features': 'sqrt', 'min_samples_leaf': 12}], 'rmse': [3.7176528093285253, 3.7107826313562025, 3.7056255569524446, 3.719016626993414, 3.7043785929107456, 3.7158577854068704], 'mae': [1.651091622713409, 1.6534310217257444, 1.6512384505671764, 1.6527416356475277, 1.6532190938779396, 1.652887767120092]}

RMSE
mean: 3.712, stdev: 0.005715
MAE
mean: 1.652, stdev:

In [1]:
k = [4]
for i in k:
    print(i)

4
